In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
인터랙티브 입출력 버전
- 실행 후 '루트 경로'와 'CSV 저장 경로'를 입력받아 두 개의 CSV(summary.csv, cases_detail.csv)를 생성함.
- 스캔 깊이: root → case(바로 아래 폴더) → series(그 아래 폴더, 깊이 +1)
- 규칙:
  * Dia만 있는 케이스: Dia_only
  * Artery_만 있는 케이스: Artery_only
  * Delay_만 있는 케이스: Delay_only
  * both: Artery_와 Delay_가 둘 다 있는 경우 (Dia 존재 여부는 무시)
  * neither: 위 어느 것도 해당 안 됨
- summary.csv:
  total_cases, Dia_only_cases, Artery_only_cases, Delay_only_cases, both_cases, total_Dia_folders
- cases_detail.csv:
  case_name, Dia_count, Artery_count, Delay_count, category
"""

import os
import csv
from pathlib import Path
from typing import List, Tuple


def list_immeDiate_subdirs(path: Path) -> List[Path]:
    if not path.is_dir():
        return []
    return [p for p in path.iterdir() if p.is_dir()]


def count_series(case_dir: Path) -> Tuple[int, int, int]:
    """case 디렉토리 바로 아래(series)에서 접두어별 갯수 합산"""
    Dia = Artery = Delay = 0
    for series in list_immeDiate_subdirs(case_dir):
        name = series.name.lower()
        if name.startswith("Dia"):
            Dia += 1
        elif name.startswith("Artery_"):
            Artery += 1
        elif name.startswith("Delay_"):
            Delay += 1
    return Dia, Artery, Delay


def categorize_case(Dia: int, Artery: int, Delay: int) -> str:
    if Artery > 0 and Delay > 0:
        return "both"
    if Artery > 0 and Delay == 0:
        return "Artery_only"
    if Delay > 0 and Artery == 0:
        return "Delay_only"
    if Artery == 0 and Delay == 0 and Dia > 0:
        return "Dia_only"
    return "neither"


def scan_root(root: Path):
    # root 바로 아래 → group 폴더들
    group_dirs = [p for p in list_immeDiate_subdirs(root)]
    case_dirs = []
    for g in group_dirs:
        case_dirs.extend(list_immeDiate_subdirs(g))  # 그룹 밑에 있는 case 폴더들 추가

    total_cases = len(case_dirs)

    Dia_only_cases = Artery_only_cases = Delay_only_cases = both_cases = 0
    total_Dia_folders = 0

    details_rows = []

    for case_dir in sorted(case_dirs, key=lambda p: p.name):
        Dia, Artery, Delay = count_series(case_dir)
        total_Dia_folders += Dia
        category = categorize_case(Dia, Artery, Delay)

        if category == "Dia_only":
            Dia_only_cases += 1
        elif category == "Artery_only":
            Artery_only_cases += 1
        elif category == "Delay_only":
            Delay_only_cases += 1
        elif category == "both":
            both_cases += 1

        details_rows.append({
            "case_name": case_dir.name,
            "Dia_count": Dia,
            "Artery_count": Artery,
            "Delay_count": Delay,
            "category": category,
        })

    summary = {
        "total_cases": total_cases,
        "Dia_only_cases": Dia_only_cases,
        "Artery_only_cases": Artery_only_cases,
        "Delay_only_cases": Delay_only_cases,
        "both_cases": both_cases,
        "total_Dia_folders": total_Dia_folders,
    }
    return summary, details_rows


def save_summary_csv(summary: dict, out_dir: Path):
    cols = [
        "total_cases",
        "Dia_only_cases",
        "Artery_only_cases",
        "Delay_only_cases",
        "both_cases",
        "total_Dia_folders",
    ]
    out_file = out_dir / "summary.csv"
    with out_file.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=cols)
        writer.writeheader()
        writer.writerow({k: summary.get(k, 0) for k in cols})
    return out_file


def save_details_csv(rows: List[dict], out_dir: Path):
    cols = ["case_name", "Dia_count", "Artery_count", "Delay_count", "category"]
    out_file = out_dir / "cases_detail.csv"
    with out_file.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=cols)
        writer.writeheader()
        writer.writerows(rows)
    return out_file


def prompt_path(prompt: str, default: str = "") -> Path:
    raw = input(f"{prompt}{' [' + default + ']' if default else ''}: ").strip().strip('"').strip("'")
    if not raw and default:
        raw = default
    return Path(raw).expanduser()


def main():
    print("=== CT 케이스 통계 CSV 생성기 ===")
    print("루트 경로 예시) D:\\CT_kidney\\thin_section_anony_complete_64.06GB")
    root_dir = prompt_path("루트 경로를 입력하세요")
    if not root_dir.exists() or not root_dir.is_dir():
        print(f"[ERROR] 디렉토리를 찾을 수 없습니다: {root_dir}")
        return
    print(root_dir)
    out_default = str(Path.cwd())
    out_dir = prompt_path("CSV 저장 경로를 입력하세요", default=out_default)
    try:
        out_dir.mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"[ERROR] 저장 경로 생성 실패: {e}")
        return

    print("\n[INFO] 스캔 중... (root → case → series)")
    summary, details = scan_root(root_dir)

    f1 = save_summary_csv(summary, out_dir)
    f2 = save_details_csv(details, out_dir)

    print("\n[DONE] CSV 생성 완료")
    print(f" - summary.csv: {f1}")
    print(f" - cases_detail.csv: {f2}")

    # 콘솔에 요약 미리보기
    print("\n[SUMMARY]")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    # 상위 10건 미리보기
    print("\n[CASES_DETAIL PREVIEW] (top 10)")
    for row in details[:10]:
        print(f"  {row['case_name']}: Dia={row['Dia_count']} / Artery={row['Artery_count']} / Delay={row['Delay_count']} -> {row['category']}")


if __name__ == "__main__":
    main()


=== CT 케이스 통계 CSV 생성기 ===
루트 경로 예시) D:\CT_kidney\thin_section_anony_complete_64.06GB
C:\Users\user\Downloads\CTkidney thin_section_anony_complete_64.06GB

[INFO] 스캔 중... (root → case → series)

[DONE] CSV 생성 완료
 - summary.csv: C:\Users\user\Downloads\summary.csv
 - cases_detail.csv: C:\Users\user\Downloads\cases_detail.csv

[SUMMARY]
  total_cases: 84
  Dia_only_cases: 0
  Artery_only_cases: 0
  Delay_only_cases: 0
  both_cases: 0
  total_Dia_folders: 0

[CASES_DETAIL PREVIEW] (top 10)
  20141225: Dia=0 / Artery=0 / Delay=0 -> neither
  20150104: Dia=0 / Artery=0 / Delay=0 -> neither
  20150225: Dia=0 / Artery=0 / Delay=0 -> neither
  20150305: Dia=0 / Artery=0 / Delay=0 -> neither
  20150430: Dia=0 / Artery=0 / Delay=0 -> neither
  20150509: Dia=0 / Artery=0 / Delay=0 -> neither
  20150516: Dia=0 / Artery=0 / Delay=0 -> neither
  20150619: Dia=0 / Artery=0 / Delay=0 -> neither
  20150717: Dia=0 / Artery=0 / Delay=0 -> neither
  20150731: Dia=0 / Artery=0 / Delay=0 -> neither
